# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Exploratory Analysis

In [3]:
# Import the CSV file
df = pd.read_csv("https://raw.githubusercontent.com/candersen114/Python2Source/main/vgsales_new_final.csv")

# Year imported as a float; convert it to an integer
#df["Year"] = df["Year"].astype("Int64")

# Scrub out records identified in Excel where the year was "N/A".
# This is a trivial number - they can be safely discarded
df = df[df["Year"].notna() == True]
print(df.dtypes)

display(df)


Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
Developer        object
Critic_Score    float64
User_Score      float64
NA_Sales        float64
PAL_Sales       float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object


,Rank,Name,Platform,Year,Genre,Publisher,Developer,Critic_Score,User_Score,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,Nintendo EAD,7.7,NaN,41.36,29.02,3.77,8.51,82.65
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,Nintendo EAD,10.0,8.2,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,Nintendo EAD,8.2,9.1,15.91,12.92,3.80,3.35,35.98
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,Nintendo EAD,8.0,8.8,15.61,10.99,3.29,3.02,32.90
4,5,Pokémon Red / Green / Blue Version,GB,1998.0,Role-Playing,Nintendo,Game Freak,9.4,NaN,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Red Dead Redemption: Undead Nightmare,X360,2010.0,Action,Rockstar Games,Rockstar San Diego,10.0,NaN,0.61,0.38,0.02,0.10,1.11
1996,1997,Metal Gear Solid HD Collection,PS3,2011.0,Action,Konami Digital Entertainment,Bluepoint Games,NaN,NaN,0.52,0.44,NaN,0.14,1.11
1997,1998,EA Sports UFC,PS4,2014.0,Fighting,Electronic Arts,EA Canada,NaN,NaN,0.49,0.43,0.01,0.18,1.11
1998,1999,NCAA Football 13,X360,2012.0,Sports,EA Sports,EA Tiburon,7.5,NaN,1.02,NaN,NaN,0.09,1.11
